In [1]:
import os
import pickle
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import time, datetime
import copy

# torch
import torch
import torch.nn as nn
import torch.optim as optim

# my own
from tasks import get_data
from myutils import *
from topology import gen_lattice, compute_dist
from scipy.stats import expon, multivariate_normal

# increase cell width
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

dtype = torch.float
device = torch.device("cpu")

# SOME USEFUL DEFINITIONS
relu = lambda x : (x > 0) * x
def pr(h, b):
    if b > 1e2:
        return 1. * (h > 0)
    else:
        return np.exp(b * h) / (2 * np.cosh(b * h))

In [2]:
# a = multivariate_normal.rvs(mean=np.zeros(20), cov=0.1, size=5)
# y = np.random.randint(3, size = 5)
# print(torch.tensor(y[5:]))

#print(int(pars.alpha_test * pars.lN))
# a=len([1,3])
# b=[None]*a
# print(a, b)

# ['a']*2
# 1-True
# d=5
# positions = torch.arange(d, dtype=torch.float)
# distances = positions[None] - positions[:, None]
# print(positions[None])
# print(positions[:,None])
# print(distances)
# b=torch.minimum(torch.abs(distances), d- torch.abs(distances))
# print(b)

# a=np.array([[1,2,3],[3,4,5]])
# print(a.shape)
# print(np.kron(a,a))

In [5]:
# SET MAIN PARAMS

# Pars() initializes a Class with definitions for parameters
pars = Pars()

pars.dataset = "RANDOM" #"NLGP" #Non-linear Gaussian Process

## spatial options
pars.dim = 1             # Input dimension
pars.normalize = False   # Normalize inputs (some dataset are already normalized)

## random dataset options
pars.alpha_test = 0.     # Fraction of test patterns wrt N in a random task
pars.span_h = False       # Generate all h in range (only makes sense for N=2 or N=3)
pars.rho = 0.             # Input covariance

## NLGP specific options
pars.g_nonlin = 0#1e-10      # Gain parameter for nonlinearity
#pars.xis = [1e-10]           #-# Lenght-scales of mixtures (?)

## misc options
pars.seed_data = 1                   # Initialize a random seed
pars.data_dir = f'data'              # Where data will be saved (it actually is not saved)
pars.save_dir = f'results_notebook'  # Where results will be saved (they actually are not solved)

makedir(pars.data_dir)
makedir(pars.save_dir)

In [6]:
# Global network options
noself = True                      # Neurons don't link with themselves
use_bias = False                   # Use a bias/input current
learn_bias = False                 # Learn the bias
bias = 0.                          # Initial value for the bias
norm_dependent_stability = False   #-# The margin inforced for stability of the solution depends on the norm of the weights?
dim = pars.dim                     # Dimension of the space where the inputs lives in (a line, R2, R3, etc)

# learning options
num_epochs = 1000          # Number of epochs
print_every = 100          # Verbose option
lr = 0.1                  # Learning rate
delta0 = 0                #-# Strength for inforcing the margin of stability
gam = 0.                  #-# Regularization strength (for not having big weights)

In [7]:
def train_network(X, Y, N, P, noself=True, num_epochs=200, delta0=0.0, bias=0, use_bias=False, learn_bias=False, stop_at_zero_err=True):
    # init J: matrix with weight between neurons
    #J = np.random.randn(N, N) / np.sqrt(N)      # Initialize J with random values
    J = np.zeros((N,N))
    diag_indices = np.diag_indices(N)           # The diagonal indices
    if noself:                                  # Optionally, set diagonal to 0
        J[diag_indices] = 0.                    
    J0 = J.copy()                               # Save state of J before training
    
    # Init I: bias input matrix
    I = bias * np.ones(N) 
    
    # Initialize variables
    errs, en_regs, sign_regs, pos_regs = [], [], [], []  # Holders for error estimation
    ok = False                                           # Will say when to stop (at convergence) 
    
    # run optimization
    for ep in range(num_epochs):
    
        # run through data (updating units randomly)
        perm = np.random.permutation(P) # random dynamical update
        for mu in perm:
    
            # compute fields
            H = J @ X[mu]               # Weights*data
            if use_bias:                # Optionally, pdate the bias
                H += I                   
    
            # learning step
            delta = delta0 * np.sqrt((J**2).sum(-1)) if norm_dependent_stability else delta0 * np.sqrt(N)  # Margin for stability
            err = Y[mu] * ((Y[mu] * H) <= 0)                 # 
            derr = np.outer(err,X[mu])
            J += lr * derr
            #J -= lr * gam * J
    
            # train input bias
            if use_bias and learn_bias:
                I += lr * err
    
            # apply energy, sign and position regularization; enforce weight constraings
            if noself:                                  # Optionally, set diagonal to 0
                J[diag_indices] = 0.                    
        
        # compute errors and print
        if ep % print_every == 0:
    
            # compute loss on entire training set
            Hs = X @ J.T
            if use_bias:
                Hs += I[None]
            err = ((Hs * Y) <= delta).mean()
            errs += [err]
    
            # compute energy, sign and position regularization error
    
            # print info
            #toprint = f'ep: {ep} loss: {err}'
            #print(toprint)
    
            # optionally stop at convergence, print epoch of convergence
            if err.sum() == 0:
                if not ok:
                    ep_conv = ep
                    ok = True
                if stop_at_zero_err:
                    break
    
    if not ok:
        print(f'loss not zero in {ep} epochs - min av error {errs[-1]}')
        return errs, errs[-1], J, J0
    else:
        print(f'zero loss converged at {ep_conv} epochs')
        return errs, errs[-1], J, J0

In [9]:
# Train for different values of alpha
delta0 = 0#0.001
N_span=[1000, 1500]
alpha_span = [1.7, 2, 2.3]#, 2.6]#np.linspace(0.1,3,6)
for sample_dimension in N_span:#, 500, 1000, 2000]:
    print(f'######################### N={sample_dimension}')
    pars.lN = sample_dimension            # Linear input dimension
    errors = []
    for alpha_train in alpha_span:
        pars.alpha_train = alpha_train     # Fraction of training patterns wrt N in a random task
        
        # GET DATASET
        data_and_properties = get_data(pars, dtype=dtype, device=device)
        train_dataset, test_dataset, teacher_weights, x_teacher = data_and_properties
        # generatae patterns of -1s and 1s
        X = np.sign(train_dataset.tensors[0].numpy()) #0.5 * (1 + np.sign(train_dataset.tensors[0].numpy())) #
        Y = X #2*X-1#train_dataset.tensors[0].numpy()#np.sign(X)

        print(f"alpha={alpha_train}") #Verbose option
        #print(f"num train: {pars.num_train}, num test: {pars.num_test}, num label: {pars.num_label}")
        P = pars.num_train            # Number of inputs 
        #print(P/N)
        N = pars.N       # linear and total input dimension
    
        errs, err, J,J0 = train_network(X, Y, N, P, noself=True, num_epochs=num_epochs, delta0=delta0, 
                                    use_bias=use_bias, bias=0, learn_bias=learn_bias, stop_at_zero_err=True)
        errors.append(err)
    
    plt.plot(alpha_span, errors, label=N)
    plt.xlabel('alpha')
    plt.ylabel('error')
    plt.title(f'Critcal capacity for N up to {N}')
    plt.savefig(f'{pars.save_dir}/Critical-{N}')
    plt.legend()
    #plt.close()


######################### N=1000
alpha=1.7
loss not zero in 999 epochs - min av error 4.823529411764706e-05
alpha=2
loss not zero in 999 epochs - min av error 0.1311065
alpha=2.3


KeyboardInterrupt: 

In [ ]:
h=J@X[1]
err=X[1]*((Y[1]*h) <= 0)
print(err)
print(X[1])
print(np.outer(err,X[1]))

In [ ]:
for m in range(int(alpha_train*N)):
    incorrect = (np.sign(J@X[m])!=X[m]).sum(-1)
    if incorrect >0:
        print(incorrect)

J

In [ ]:
plt.plot(errs)